<a href="https://colab.research.google.com/github/kei-mo/bayesian-ml/blob/main/chapter4_gibbs_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter4.2.2 ギブスサンプリング  



## 問題背景
ベイズ推論：データを説明するモデルのパラメータを分布として推定する   
* 共役関係にある確率分布を使う場合→解析的に確率推論を実施することができる。  
* 複雑なモデル(ex. 混合モデル)や共役関係にない確率分布を扱う場合  
→解析的に確率推論を実施できない。  
→**確率分布を近似して推論する**。


## ベイズ推定での近似推論方法  
* Markov Chain Monte Carlo (MCMC)法  
* 変分推論

### MCMC

乱数を生成して
* メトロポリス法
* ギブスサンプラー
* ハミルトニアン・モンテカルロ法

### 変分推論
最適化問題を解いて未知の確率分布の近似的な表現を得る  

## ギブスサンプリング



1. $z_1^{i=0}$, $z_2^{i=0}$, $z_3^{i=0}$を初期化   
*for i in loop*
2. $z_1^{i} \sim p(z_1^{i}| z_2^{i-1}, z_3^{i-1}$)  
$z_2^{i-1}$, $z_3^{i-1}$に基づいて$p(z_1^{i}| z_2^{i-1}, z_3^{i-1}$)から$z_1^{i}$をサンプリング　($z_2^{i-1}$, $z_3^{i-1}$は固定)
3. $z_2^{i} \sim p(z_2^{i}| z_1^{i}, z_3^{i-1}$)
4. $z_3^{i}\sim p(z_3^{i}| z_1^{i}, z_2^{i}$)



### 2次元ガウス分布の例

In [1]:
# import some common libraries
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def gaussian2d(z0,z1,mean,co_var):
    """2D Gaussian distribution
    
    """


In [ ]:
# setup params
a = 0.5
step = 100


# initialize
z0 = 0
z1 = 0
list_sample = []

for i in range(step):




## 混合ガウス分布の例